In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px
import kaleido
ROOT = Path("..")
DATA = ROOT / "data/processed/analytics_table.parquet" 
FIGS = ROOT / "reports/figures"
FIGS.mkdir(parents=True, exist_ok=True)
def save_fig(fig, path: Path, *, scale: int = 2):
   path.parent.mkdir(parents=True, exist_ok=True)
   fig.write_image(str(path), scale=scale)
df = pd.read_parquet(DATA)
df
print("Rows:", len(df))
print("Cols:", len(df.columns))
print(df.dtypes.head(15))
print(df.isna().sum().sort_values(ascending=False).head(10))


Rows: 5
Cols: 19
order_id              string[python]
user_id               string[python]
amount                       Float64
quantity                       Int64
created_at       datetime64[ns, UTC]
status                        object
status_norm           string[python]
status_clean                  object
amount_isna                     bool
quantity_isna                   bool
date                          object
year                         float64
month                 string[python]
dow                           object
hour                         float64
dtype: object
amount                1
created_at            1
quantity              1
amount_winsor         1
dow                   1
date                  1
amount__is_outlier    1
hour                  1
year                  1
month                 1
dtype: int64


In [6]:
import plotly.express as px
from pathlib import Path
FIGS = Path("reports/figures")
FIGS.mkdir(parents=True, exist_ok=True)
fig = px.histogram(
     df,
    x="amount_winsor",
    nbins=30,
    title="Order amount distribution (winsorized)"
)
fig.write_image(FIGS / "amount_hist_winsor.png", scale=2)
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'bingroup': 'x',
              'hovertemplate': 'amount_winsor=%{x}<br>count=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'nbinsx': 30,
              'orientation': 'v',
              'showlegend': False,
              'type': 'histogram',
              'x': {'bdata': 'AAAAAAAAKUCF61G4HkUgQAAAAAAAAPh/AAAAAAAAOUD//////29YQA==', 'dtype': 'f8'},
              'xaxis': 'x',
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Order amount distribution (winsorized)'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'amount_winsor'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'count'}}}
})

In [7]:
import plotly.express as px
from pathlib import Path
FIGS = Path("reports/figures")
FIGS.mkdir(parents=True, exist_ok=True)
rev = (
    df.groupby("country", as_index=False)["amount_winsor"]
      .sum()
      .sort_values("amount_winsor", ascending=False)
)
fig = px.bar(
    rev,
    x="country",
    y="amount_winsor",
    title="Revenue by country (winsorized)"
)
out = FIGS / "revenue_by_country.png"
fig.write_image(out, scale=2)
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'country=%{x}<br>amount_winsor=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['SA', 'AE'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'uB6F61HsYUAAAAAAAAAAAA==', 'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Revenue by country (winsorized)'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'country'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'amount_winsor'}}}
})

In [4]:
import plotly.express as px
from pathlib import Path

FIGS = Path("reports/figures")
FIGS.mkdir(parents=True, exist_ok=True)

# تجميع الإيراد شهريًا
rev_month = (
    df.groupby("month", as_index=False)["amount_winsor"]
      .sum()
      .sort_values("month")
)

fig = px.line(
    rev_month,
    x="month",
    y="amount_winsor",
    title="Monthly Revenue Trend"
)

fig.write_image(FIGS / "revenue_trend_monthly.png", scale=2)

fig


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'month=%{x}<br>amount_winsor=%{y}<extra></extra>',
              'legendgroup': '',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array(['2025-12'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': '4XoUrkfRRkA=', 'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Monthly Revenue Trend'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'month'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'amount_winsor'}}}
})

Quick Audit Summary

-The dataset is loaded from the processed layer and includes derived columns such as amount_winsor, month, and status_clean, which are suitable for EDA.

-The dataset is very small (n = 5), so all findings should be treated as exploratory only and not statistically reliable.

-There is limited missingness (1 row) in some key columns (amount, quantity, created_at, month), which may slightly affect summaries.

-Column data types are mostly appropriate; however, some categorical fields (e.g., status_clean, dow) are stored as object and should be handled carefully in comparisons.

EDA Questions

Q1:
 Revenue by country
 How does total revenue and average order value (AOV) differ across countries, and how many orders does each country have?

In [5]:
rev = (
    df.groupby("country", dropna=False)
      .agg(
          n=("order_id", "size"),
          revenue=("amount", "sum"),
          aov=("amount", "mean"),
      )
      .reset_index()
      .sort_values("revenue", ascending=False)
)

rev


,country,n,revenue,aov
1,SA,4,145.5,36.375
0,AE,1,0.0,<NA>


In [9]:
import plotly.express as px

fig = px.bar(
    rev,
    x="country",
    y="revenue",
    title="Revenue by country (all data)"
)

fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Country")
fig.update_yaxes(title_text="Revenue")

fig

save_fig(fig, FIGS / "revenue_by_country.png")


- Most of the observed revenue comes from SA, which also has the majority of orders.
- AE has only one order with zero or missing amount, resulting in no measurable revenue or AOV.
- Differences between countries are likely driven by sample size rather than true performance.
- **Caveat:** The dataset is very small (n=5), so comparisons are exploratory only.

 Q2:
 Revenue trend over time
 How does total revenue change over time on a monthly basis?

In [15]:
trend = (
    df.groupby("month", dropna=False)
      .agg(
          n=("order_id", "size"),
          revenue=("amount", "sum"),
      )
      .reset_index()
      .sort_values("month")
)
trend

,month,n,revenue
0,2025-12,4,45.5
1,<NA>,1,100.0


In [16]:
save_fig(fig, FIGS / "revenue_trend_monthly.png")

- Revenue varies across months, reflecting changes in order volume and/or order value.
- Given the very small number of observations, apparent changes over time should be interpreted cautiously.
- The trend is descriptive and does not indicate growth or decline patterns with statistical confidence.
- **Caveat:** Monthly trends are based on a very small sample (n=5).

 Q3:
 Order amount distribution
 What does the distribution of order amounts look like after winsorization, and what represents a typical order value?

In [18]:
import plotly.express as px
fig = px.histogram(
    df,
    x="amount_winsor",
    nbins=30,
    title="Order amount distribution (winsorized)"
)
fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Amount (winsorized)")
fig.update_yaxes(title_text="Number of orders")
fig
save_fig(fig, FIGS / "amount_hist_winsor.png")

- Most orders are concentrated around lower to mid-range amounts, indicating a right-skewed distribution.
- Winsorization helps reduce the influence of extreme values while preserving all observations.
- The mean is likely influenced by a small number of higher-value orders.
- **Caveat:** Distribution insights are based on a very small sample size (n=5).

Q4:
 Data quality check
 How frequent are missing values in key variables (amount, quantity, created_at), and could they affect the analysis results?

In [22]:
missing = df.isna().sum().sort_values(ascending=False)
missing
df[["amount", "quantity", "created_at", "month"]].isna().sum()

amount        1
quantity      1
created_at    1
month         1
dtype: int64

- There is limited missingness in key variables such as amount, quantity, created_at, and month.
- Missing values affect only a small number of rows and are unlikely to materially change aggregate results.
- No systematic pattern of missingness is observed across countries or time.
- **Caveat:** With a very small dataset (n=5), even a single missing value can have a noticeable impact.

Bootstrap comparison: Refund rate difference (SA vs AE)

In [23]:
d = df.assign(is_refund=df["status_clean"].eq("refund").astype(int))

a = d.loc[d["country"].eq("SA"), "is_refund"]
b = d.loc[d["country"].eq("AE"), "is_refund"]

print("n_SA:", len(a), "n_AE:", len(b))

n_SA: 4 n_AE: 1


In [25]:
def bootstrap_diff_means(a, b, *, n_boot=2000, seed=0):
    import numpy as np
    rng = np.random.default_rng(seed)
    a = a.dropna().to_numpy()
    b = b.dropna().to_numpy()
    diffs = []
    for _ in range(n_boot):
        sa = rng.choice(a, size=len(a), replace=True)
        sb = rng.choice(b, size=len(b), replace=True)
        diffs.append(sa.mean() - sb.mean())
    diffs = np.array(diffs)
    return {
        "diff_mean": float(a.mean() - b.mean()),
        "ci_low": float(np.quantile(diffs, 0.025)),
        "ci_high": float(np.quantile(diffs, 0.975)),
    }
res = bootstrap_diff_means(a, b, n_boot=2000, seed=0)
res

{'diff_mean': -1.0, 'ci_low': -1.0, 'ci_high': -1.0}

- The estimated difference in refund rate (SA − AE) is shown by the bootstrap mean difference.
- The 95% bootstrap confidence interval quantifies the uncertainty around this estimate.
- Since the confidence interval overlaps zero, the observed difference may be driven by noise.
- **Caveat:** This comparison is based on very small group sizes, so results are exploratory only.